# Pytorch를 활용한 Sign Language 검출

In [3]:
import pandas as pd
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import random
import os
import time
import copy
import glob
from tqdm import tqdm as tqdmd

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
from torchvision import transforms
from torchvision.io import read_image
import torchvision.models as models
from torchsummary import summary


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Using matplotlib backend: Qt5Agg


## Dataset Preview

![ASL](./image/01.png)
* ASL(American Sign Language) alphabet은 총 26개가 있지만 dataset에서는 25개 밖에 없다. 
* 그 이유는 Z가 검지로 Z를 그리는 것이기 때문에 사진으로는 표현이 불가능.

## Image dir, path

In [2]:
TRAIN_DIR = "Train/"
TEST_DIR = "Test/"
TRAIN_DIR_LIST = os.listdir(TRAIN_DIR)
TEST_DIR_LIST = os.listdir(TEST_DIR)

In [71]:
#Trian Dataset
df = pd.DataFrame(columns = ['label', 'dir'])
for i in TRAIN_DIR_LIST:
    for j in os.listdir(TRAIN_DIR + i +"/"):
        data = {'label' : i, 'dir' : TRAIN_DIR + i + "/" + j }
        df = df.append(data, ignore_index=True)
df.to_csv("Train/train.csv")

In [72]:
df = pd.DataFrame(columns = ['label', 'dir'])
for i in TEST_DIR_LIST:
    for j in os.listdir(TEST_DIR + i +"/"):
        data = {'label' : i, 'dir' : TEST_DIR + i + "/" + j }
        df = df.append(data, ignore_index=True)
df.to_csv("Test/test.csv")

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7172 entries, 0 to 7171
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7172 non-null   object
 1   dir     7172 non-null   object
dtypes: object(2)
memory usage: 112.2+ KB


In [22]:
train_dataset = pd.read_csv("Train/train.csv", index_col = 0)
test_dataset = pd.read_csv("Test/test.csv", index_col = 0)

In [23]:
train_dataset.head(5)

,label,dir
0,A,Train/A/10014_A.jpg
1,A,Train/A/10021_A.jpg
2,A,Train/A/10023_A.jpg
3,A,Train/A/10050_A.jpg
4,A,Train/A/10056_A.jpg


In [6]:
train_dataset.label.value_counts()

R    1294
Q    1279
L    1241
W    1225
F    1204
S    1199
D    1196
O    1196
T    1186
X    1164
I    1162
U    1161
N    1151
C    1144
A    1126
Y    1118
K    1114
G    1090
P    1088
V    1082
M    1055
H    1013
B    1010
E     957
Name: label, dtype: int64

## Data Check

In [7]:
train_dataset.iloc[:, 0]

0        A
1        A
2        A
3        A
4        A
        ..
27450    Y
27451    Y
27452    Y
27453    Y
27454    Y
Name: label, Length: 27455, dtype: object

In [8]:
image = Image.open(train_dataset.loc[0][1])

In [9]:
image.size

(28, 28)

In [10]:
image

## Mapping
* A ~ Y -> 0 ~ 24

In [11]:
mapping = {}
for i in range(25):
    mapping[chr(65+i)] = i

In [27]:
train_dataset = train_dataset.replace(mapping)
test_dataset = test_dataset.replace(mapping)

In [28]:
train_dataset.label.value_counts()

17    1294
16    1279
11    1241
22    1225
5     1204
18    1199
14    1196
3     1196
19    1186
23    1164
8     1162
20    1161
13    1151
2     1144
0     1126
24    1118
10    1114
6     1090
15    1088
21    1082
12    1055
7     1013
1     1010
4      957
Name: label, dtype: int64

## transformer
* RandomHorizontalFlip(p)


Horizontally flip the given image randomly with a given probability.
* RandomRotation(degrees)


Rotate the image by angle.
* RandomVerticalFlip(p)


Vertically flip the given image randomly with a given probability.



In [14]:
DIM = (28, 28)
NUM_CLASSES = 25
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#color img
#MEAN = [0.485, 0.456, 0.406]
#STD = [0.229, 0.224, 0.225]

#gray img
MEAN = [0.5]
STD = [0.5]

In [15]:
def transform_data(value = 'val'):
    if value == 'train':
            transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(DIM),
            transforms.RandomHorizontalFlip(p = 0.5),
            transforms.RandomRotation(90),
            transforms.RandomVerticalFlip(p = 0.5),
            transforms.ToTensor(),
            transforms.Normalize(MEAN, STD)
        ])
            return transform
    
    elif value == 'val':
            transform = torchvision.transforms.Compose([
            torchvision.transforms.ToPILImage(),
            torchvision.transforms.Resize(DIM),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(MEAN, STD)
        ])
            return transform

## DataLoader

In [16]:
class ASLDataset(Dataset):
    
    def __init__(self, img_ids, labels, transform=None):
        self.labels = labels
        self.file_dir = img_ids
        self.transform = transform
        
    
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img = read_image(os.path.join(self.file_dir[idx]))
        
        if self.transform:
            img = self.transform(img)

        label = torch.tensor(self.labels[idx], dtype = torch.long)
        # image, label을 return
        return img, label

## Model

RestNet-152
![ASL](./image/02.png)


In [ ]:
def getModel():
    net = models.resnet152(pretrained=True)
    
    # if you want to train the whole network, comment this code
    # freeze all the layers in the network
    for param in net.parameters():
        param.requires_grad = False
        
    num_ftrs = net.fc.in_features
    # create last few layers
    net.fc = nn.Sequential(
        nn.Linear(num_ftrs, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, NUM_CLASSES),
        nn.LogSoftmax(dim=1)
    )
    
    # use gpu if any
    net = net.cuda() if DEVICE else net
    return net
ASL_classification = getModel()

## Model Summary

In [ ]:
summary(cassava_classification, input_size = (1,28, 28))

## Cyclical Learning Rates

In [ ]:
import math
def cyclical_lr(stepsize, min_lr=3e-4, max_lr=3e-3):

    # Scaler: we can adapt this if we do not want the triangular CLR
    scaler = lambda x: 1.

    # Lambda function to calculate the LR
    lr_lambda = lambda it: min_lr + (max_lr - min_lr) * relative(it, stepsize)

    # Additional function to see where on the cycle we are
    def relative(it, stepsize):
        cycle = math.floor(1 + it / (2 * stepsize))
        x = abs(it / stepsize - 2 * cycle + 1)
        return max(0, (1 - x)) * scaler(cycle)

    return lr_lambda

## Loss Function, Optimizer, scheduler

#### torch.nn.CrossEntropyLoss
![Loss](./image/03.png)

#### torch.optim.Adam

__1. 개요__

model을 학습시키기 위해선 optimization라는 작업을 해줌. weight의 torch.backward()라는 함수로, gradient를 구해서 weight의 값을 변화시켜주는 역활을 함.

__2. Adam__
![Adam](./image/04.png)
Adam method는 Adagrad + RMSProp의 장점을 섞어 놓은 것으로 __stepsize가 gradient의 rescaling에 영향을 받지 않음.__ 즉, gradient가 커져도 stepsize는 bound되어 있어서 어떠한 object function을 사용하더라도 안정적으로 optimization을 위한 하강이 가능하다.

Adam pseudo-code
![Adama_pesudo_code](./image/05.png)

* AdaGrad란?


AgaGrad는 과거의 gradient 변화량을 참고한다. 예를 들어, 이미 많이 변화한 변수들은 optimum에 거의 도달했다 보고 stepsize를 작게하는 반면, 여태까지 많이 변화하지 않은 변수들은 optimum에 도달하지 않았다고 보고 stepsize를 크게 한다.
![AdaGrad](./image/06.png)
하지만 iteration이 계속될 수록 stepsize가 너무 작아질 수 있는 문제점이 존재한다.

* RMSProp란?


AdaGrad의 stepsize와 관련된 문제점을 해결한 것이 RMSProp이다. RMSProp은 [exponential moving average](https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp)를 사용하여 과거의 정보에 가중치를 작게, 최근 값에 민감하도록 가중치를 크게 부여하는 형태이다.
![AdaGrad](./image/07.png)
하지만 G 변수의 bias를 수정해주지 않으면 B2가 1에 가까워질수록 매우 큰 stepsize를 갖거나 발산하게 되는 문제점이 있다.

#### learning rate scheduler 
learnig rate는 gradient의 보폭을 말한다. 처음부터 같은 learning rate를 사용할 수 있지만, 학습 과정에서 learning rate를 조정하는 scheduler을 사용할 수 있다. 처음에 큰 learning rate로 빠르게 optimize를 하고 최적값에 가까워질수록 learning rate를 줄이는 것이 더 효과적이다.
![lr](./image/08.png)

#### LambdaLR
Lambda 표현식으로 작성한 함수를 통해 lr을 조정함. 초기 learning rate에 lambda함수에서 나온 값을 곱해줘서 lr을 계산.
![lambdalr](./image/09.png)


아래의 그래프처럼 learning rate가 조정된다.(개인적인 생각이지만 고정된 값으로 감소하기 때문에 효과가 좋을까? 의문임.. 다른 방법도 찾아봐서 채택해도 괜찮을듯)
![lambdalr_graph](./image/10.png)

In [ ]:
criterion = nn.CrossEntropyLoss()
model     = ASL_classification
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.95 ** epoch)

## Create DataLoad for Train, Test

In [17]:
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
NUM_WORKERS = 2

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_dataset['dir'], train_dataset['label'], test_size=0.25)

In [31]:
train = ASLDataset(x_train.values, y_train.values,
                   transform = transform_data("train"))

train_loader = DataLoader(train,
                        batch_size=TRAIN_BATCH_SIZE,
                        num_workers=NUM_WORKERS,
                        shuffle=False)



test = ASLDataset(x_train.values, y_train.values,
                  transform = transform_data("val"))

test_loader = DataLoader(test,
                         batch_size=TEST_BATCH_SIZE,
                        num_workers=NUM_WORKERS,
                        shuffle=False)

In [ ]:
loaders = {'train': train_loader, 'val': test_loader}

## Step for Train and Test

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10, scheduler=scheduler):
    start_time = time.time()
    
    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*15)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() 
            else:
                model.eval() 
                
            running_loss = 0.0
            running_corrects = 0
            fin_out = []
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # reset (or) zero the parameter gradients
                optimizer.zero_grad()
                
                # training (or) validation process
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    
                    _, preds = torch.max(outputs, 1)
                    
                    # 역전파(back propagation) 
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            # accuarcy for the epoch
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            # update the best weights
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                
        print()
    end_time = time.time() - start_time
    
    print('Training completes in {:.0f}m {:.0f}s'.format(end_time // 60, end_time % 60))
    print('Best Val Acc: {:.4f}'.format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

            

## Model Save and Predict

In [ ]:
torch.save(model, 'best_model.h5')
torch.save(model.state_dict(), 'best_model_weights')

In [ ]:
unfreeze = True 
for param in model.parameters():
    param.requires_grad = unfreeze

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters')
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{trainable_params:,} training parameters')

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    fin_out = []
    
    with torch.no_grad():
        images = images.to(device)
        targets = targets.to(device)
            
        outputs = model(images)
            
        fin_out.append(F.softmax(outputs, dim=1).detach().cpu().numpy())        
    return np.concatenate(fin_out)
